# Balanced sampling

This module offers a number of re-sampling techniques including under-sampling majority classes, over-sampling minority classes, and combinations of the two.

Balanced sampling was added in MADlib 1.14.

In [2]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [3]:
# Greenplum Database 5.4.0 on GCP (demo machine)
%sql postgresql://gpadmin@35.184.253.255:5432/madlib
        
# PostgreSQL local
#%sql postgresql://fmcquillan@localhost:5432/madlib

# Greenplum Database 4.3.10.0
#%sql postgresql://gpdbchina@10.194.10.68:61000/madlib

u'Connected: gpadmin@madlib'

In [4]:
%sql select madlib.version();
#%sql select version();

1 rows affected.


version
"MADlib version: 1.14-dev, git revision: rc/1.13-rc1-22-g0bfcaf5, cmake configuration time: Wed Mar 14 21:35:16 UTC 2018, build type: release, build system: Linux-2.6.32-696.20.1.el6.x86_64, C compiler: gcc 4.4.7, C++ compiler: g++ 4.4.7"


# 1. Load data
Based in part on the flags data set from https://archive.ics.uci.edu/ml/datasets/Flags

In [5]:
%%sql
DROP TABLE IF EXISTS flags;

CREATE TABLE flags (
    id INTEGER,
    name TEXT,
    landmass INTEGER,
    zone INTEGER,
    area INTEGER,
    population INTEGER,
    language INTEGER,
    colours INTEGER,
    mainhue TEXT
);

INSERT INTO flags VALUES
(1, 'Argentina', 2, 3, 2777, 28, 2, 2, 'blue'),
(2, 'Australia', 6, 2, 7690, 15, 1, 3, 'blue'),
(3, 'Austria', 3, 1, 84, 8, 4, 2, 'red'),
(4, 'Brazil', 2, 3, 8512, 119, 6, 4, 'green'),
(5, 'Canada', 1, 4, 9976, 24, 1, 2, 'red'),
(6, 'China', 5, 1, 9561, 1008, 7, 2, 'red'),
(7, 'Denmark', 3, 1, 43, 5, 6, 2, 'red'),
(8, 'Greece', 3, 1, 132, 10, 6, 2, 'blue'),
(9, 'Guatemala', 1, 4, 109, 8, 2, 2, 'blue'),
(10, 'Ireland', 3, 4, 70, 3, 1, 3, 'white'),
(11, 'Jamaica', 1, 4, 11, 2, 1, 3, 'green'),
(12, 'Luxembourg', 3, 1, 3, 0, 4, 3, 'red'),
(13, 'Mexico', 1, 4, 1973, 77, 2, 4, 'green'),
(14, 'Norway', 3, 1, 324, 4, 6, 3, 'red'),
(15, 'Portugal', 3, 4, 92, 10, 6, 5, 'red'),
(16, 'Spain', 3, 4, 505, 38, 2, 2, 'red'),
(17, 'Sweden', 3, 1, 450, 8, 6, 2, 'blue'),
(18, 'Switzerland', 3, 1, 41, 6, 4, 2, 'red'),
(19, 'UK', 3, 4, 245, 56, 1, 3, 'red'),
(20, 'USA', 1, 4, 9363, 231, 1, 3, 'white'),
(21, 'xElba', 3, 1, 1, 1, 6, NULL, NULL),
(22, 'xPrussia', 3, 1, 249, 61, 4, NULL, NULL);

SELECT * FROM flags ORDER BY mainhue, name;

Done.
Done.
22 rows affected.
22 rows affected.


id,name,landmass,zone,area,population,language,colours,mainhue
1,Argentina,2,3,2777,28,2,2,blue
2,Australia,6,2,7690,15,1,3,blue
8,Greece,3,1,132,10,6,2,blue
9,Guatemala,1,4,109,8,2,2,blue
17,Sweden,3,1,450,8,6,2,blue
4,Brazil,2,3,8512,119,6,4,green
11,Jamaica,1,4,11,2,1,3,green
13,Mexico,1,4,1973,77,2,4,green
3,Austria,3,1,84,8,4,2,red
5,Canada,1,4,9976,24,1,2,red


# 2.  Uniform sampling 

All class values will be resampled so that they have the same number of rows. The output data size will be the same as the input data size, ignoring NULL values.  Uniform sampling is the default for the 'class_size' parameter so we do not need to explicitly set it:  

In [6]:
%%sql
DROP TABLE IF EXISTS output_table;

SELECT madlib.balance_sample(
                              'flags',             -- Source table
                              'output_table',      -- Output table
                              'mainhue');          -- Class column
  
SELECT * FROM output_table ORDER BY mainhue, name;

Done.
1 rows affected.
20 rows affected.


__madlib_id__,id,name,landmass,zone,area,population,language,colours,mainhue
19,1,Argentina,2,3,2777,28,2,2,blue
18,2,Australia,6,2,7690,15,1,3,blue
20,8,Greece,3,1,132,10,6,2,blue
17,9,Guatemala,1,4,109,8,2,2,blue
16,17,Sweden,3,1,450,8,6,2,blue
6,4,Brazil,2,3,8512,119,6,4,green
7,4,Brazil,2,3,8512,119,6,4,green
9,13,Mexico,1,4,1973,77,2,4,green
10,13,Mexico,1,4,1973,77,2,4,green
8,13,Mexico,1,4,1973,77,2,4,green


Next we do uniform sampling again, but this time we specify a size for the output table:

In [7]:
%%sql
DROP TABLE IF EXISTS output_table;

SELECT madlib.balance_sample(
                              'flags',             -- Source table
                              'output_table',      -- Output table
                              'mainhue',           -- Class column
                              'uniform',           -- Uniform sample
                               12);                -- Desired output table size

SELECT * FROM output_table ORDER BY mainhue, name;

Done.
1 rows affected.
12 rows affected.


__madlib_id__,id,name,landmass,zone,area,population,language,colours,mainhue
3,8,Greece,3,1,132,10,6,2,blue
2,9,Guatemala,1,4,109,8,2,2,blue
1,17,Sweden,3,1,450,8,6,2,blue
4,4,Brazil,2,3,8512,119,6,4,green
6,11,Jamaica,1,4,11,2,1,3,green
5,13,Mexico,1,4,1973,77,2,4,green
9,5,Canada,1,4,9976,24,1,2,red
8,12,Luxembourg,3,1,3,0,4,3,red
7,18,Switzerland,3,1,41,6,4,2,red
12,10,Ireland,3,4,70,3,1,3,white


# 3. Oversampling
Oversample with replacement such that all class values except NULLs end up with the same number of observations as the majority class.  Countries with red flags is the majority class with 10 observations, so other class values will be oversampled to 10 observations:

In [8]:
%%sql
DROP TABLE IF EXISTS output_table;

SELECT madlib.balance_sample(
                              'flags',             -- Source table
                              'output_table',      -- Output table
                              'mainhue',           -- Class column
                              'oversample');       -- Oversample

SELECT * FROM output_table ORDER BY mainhue, name;

Done.
1 rows affected.
40 rows affected.


__madlib_id__,id,name,landmass,zone,area,population,language,colours,mainhue
37,2,Australia,6,2,7690,15,1,3,blue
39,2,Australia,6,2,7690,15,1,3,blue
38,2,Australia,6,2,7690,15,1,3,blue
40,8,Greece,3,1,132,10,6,2,blue
32,9,Guatemala,1,4,109,8,2,2,blue
31,9,Guatemala,1,4,109,8,2,2,blue
35,17,Sweden,3,1,450,8,6,2,blue
33,17,Sweden,3,1,450,8,6,2,blue
36,17,Sweden,3,1,450,8,6,2,blue
34,17,Sweden,3,1,450,8,6,2,blue


# 4. Undersampling
Undersample such that all class values except NULLs end up with the same number of observations as the minority class.  Countries with white flags is the minority class with 2 observations, so other class values will be undersampled to 2 observations:

In [9]:
%%sql
DROP TABLE IF EXISTS output_table;

SELECT madlib.balance_sample(
                              'flags',             -- Source table
                              'output_table',      -- Output table
                              'mainhue',           -- Class column
                              'undersample');      -- Undersample

SELECT * FROM output_table ORDER BY mainhue, name;

Done.
1 rows affected.
8 rows affected.


__madlib_id__,id,name,landmass,zone,area,population,language,colours,mainhue
2,1,Argentina,2,3,2777,28,2,2,blue
1,9,Guatemala,1,4,109,8,2,2,blue
4,4,Brazil,2,3,8512,119,6,4,green
3,11,Jamaica,1,4,11,2,1,3,green
6,14,Norway,3,1,324,4,6,3,red
5,19,UK,3,4,245,56,1,3,red
7,10,Ireland,3,4,70,3,1,3,white
8,20,USA,1,4,9363,231,1,3,white


In the case of bootstrapping, we may want to undersample with replacement, so we set the 'with_replacement' parameter to TRUE:

In [10]:
%%sql
DROP TABLE IF EXISTS output_table;

SELECT madlib.balance_sample(
                              'flags',             -- Source table
                              'output_table',      -- Output table
                              'mainhue',           -- Class column
                              'undersample',       -- Undersample
                               NULL,               -- Output table size will be calculated
                               NULL,               -- No grouping
                              'TRUE');             -- Sample with replacement

SELECT * FROM output_table ORDER BY mainhue, name;

Done.
1 rows affected.
8 rows affected.


__madlib_id__,id,name,landmass,zone,area,population,language,colours,mainhue
7,9,Guatemala,1,4,109,8,2,2,blue
8,17,Sweden,3,1,450,8,6,2,blue
2,4,Brazil,2,3,8512,119,6,4,green
1,4,Brazil,2,3,8512,119,6,4,green
4,3,Austria,3,1,84,8,4,2,red
3,7,Denmark,3,1,43,5,6,2,red
6,10,Ireland,3,4,70,3,1,3,white
5,20,USA,1,4,9363,231,1,3,white


Note above that some rows may appear multiple times above since we sampled with replacement.

# 5. Setting class size by count

Here we set the number of rows for red and blue flags, and leave green and white flags unchanged:

In [11]:
%%sql
DROP TABLE IF EXISTS output_table;

SELECT madlib.balance_sample(
                              'flags',             -- Source table
                              'output_table',      -- Output table
                              'mainhue',           -- Class column
                              'red=7, blue=7');    -- Want 7 reds and 7 blues

SELECT * FROM output_table ORDER BY mainhue, name;

Done.
1 rows affected.
19 rows affected.


__madlib_id__,id,name,landmass,zone,area,population,language,colours,mainhue
11,1,Argentina,2,3,2777,28,2,2,blue
12,1,Argentina,2,3,2777,28,2,2,blue
10,1,Argentina,2,3,2777,28,2,2,blue
16,8,Greece,3,1,132,10,6,2,blue
14,9,Guatemala,1,4,109,8,2,2,blue
13,9,Guatemala,1,4,109,8,2,2,blue
15,17,Sweden,3,1,450,8,6,2,blue
17,4,Brazil,2,3,8512,119,6,4,green
8,11,Jamaica,1,4,11,2,1,3,green
9,13,Mexico,1,4,1973,77,2,4,green


Next we set the number of rows for red and blue flags, and also set an output table size.  This means that green and white flags will be uniformly sampled to get to the desired output table size:

In [12]:
%%sql
DROP TABLE IF EXISTS output_table;

SELECT madlib.balance_sample(
                              'flags',             -- Source table
                              'output_table',      -- Output table
                              'mainhue',           -- Class column
                              'red=7, blue=7',     -- Want 7 reds and 7 blues
                               22);                -- Desired output table size

SELECT * FROM output_table ORDER BY mainhue, name;

Done.
1 rows affected.
22 rows affected.


__madlib_id__,id,name,landmass,zone,area,population,language,colours,mainhue
21,2,Australia,6,2,7690,15,1,3,blue
22,8,Greece,3,1,132,10,6,2,blue
16,9,Guatemala,1,4,109,8,2,2,blue
17,9,Guatemala,1,4,109,8,2,2,blue
19,17,Sweden,3,1,450,8,6,2,blue
20,17,Sweden,3,1,450,8,6,2,blue
18,17,Sweden,3,1,450,8,6,2,blue
10,4,Brazil,2,3,8512,119,6,4,green
9,4,Brazil,2,3,8512,119,6,4,green
11,4,Brazil,2,3,8512,119,6,4,green


# 6.  NULL handling

To make NULL a valid class value, set the parameter to keep NULLs:

In [13]:
%%sql
DROP TABLE IF EXISTS output_table;

SELECT madlib.balance_sample(
                              'flags',             -- Source table
                              'output_table',      -- Output table
                              'mainhue',           -- Class column
                               NULL,               -- Uniform
                               NULL,               -- Output table size same as input
                               NULL,               -- No grouping
                               NULL,               -- Sample without replacement
                              'TRUE');             -- Make NULLs a valid class value

SELECT * FROM output_table ORDER BY mainhue, name;

Done.
1 rows affected.
25 rows affected.


__madlib_id__,id,name,landmass,zone,area,population,language,colours,mainhue
24,1,Argentina,2,3,2777,28,2,2,blue
25,2,Australia,6,2,7690,15,1,3,blue
22,8,Greece,3,1,132,10,6,2,blue
21,9,Guatemala,1,4,109,8,2,2,blue
23,17,Sweden,3,1,450,8,6,2,blue
7,4,Brazil,2,3,8512,119,6,4,green
8,4,Brazil,2,3,8512,119,6,4,green
6,4,Brazil,2,3,8512,119,6,4,green
10,11,Jamaica,1,4,11,2,1,3,green
9,13,Mexico,1,4,1973,77,2,4,green


# 7. Grouping
To perform the balance sampling for independent groups, use the 'grouping_cols' parameter. Note below that each group (zone) has a different count of the classes (mainhue), with some groups not containing some class values.

In [14]:
%%sql
DROP TABLE IF EXISTS output_table;

SELECT madlib.balance_sample(
    'flags',          -- Source table
    'output_table',   -- Output table
    'mainhue',        -- Class column
    NULL,             -- Uniform
    NULL,             -- Output table size
    'zone'            -- Grouping by zone
);

SELECT * FROM output_table ORDER BY zone, mainhue;

Done.
1 rows affected.
23 rows affected.


__madlib_id__,id,name,landmass,zone,area,population,language,colours,mainhue
1,17,Sweden,3,1,450,8,6,2,blue
3,8,Greece,3,1,132,10,6,2,blue
2,8,Greece,3,1,132,10,6,2,blue
4,8,Greece,3,1,132,10,6,2,blue
8,12,Luxembourg,3,1,3,0,4,3,red
6,18,Switzerland,3,1,41,6,4,2,red
7,14,Norway,3,1,324,4,6,3,red
5,6,China,5,1,9561,1008,7,2,red
1,2,Australia,6,2,7690,15,1,3,blue
1,1,Argentina,2,3,2777,28,2,2,blue


Grouping can be used with class size specification as well. Note below that 'blue=<Integer>' is the only valid class value since 'blue' is the only class value that is present in each group. Further, blue=8` in the example below will be split between the four groups, resulting in two blue rows for each group.

In [15]:
%%sql
DROP TABLE IF EXISTS output_table;

SELECT madlib.balance_sample(
    'flags',          -- Source table
    'output_table',   -- Output table
    'mainhue',        -- Class column
    'blue=8',         -- Specified class value size. Rest of the values are outputed as is.
    NULL,             -- Output table size
    'zone'            -- Group by zone
);

SELECT * FROM output_table ORDER BY zone, mainhue;

Done.
1 rows affected.
23 rows affected.


__madlib_id__,id,name,landmass,zone,area,population,language,colours,mainhue
3,17,Sweden,3,1,450,8,6,2,blue
4,8,Greece,3,1,132,10,6,2,blue
2,7,Denmark,3,1,43,5,6,2,red
7,14,Norway,3,1,324,4,6,3,red
8,18,Switzerland,3,1,41,6,4,2,red
6,12,Luxembourg,3,1,3,0,4,3,red
1,3,Austria,3,1,84,8,4,2,red
5,6,China,5,1,9561,1008,7,2,red
1,2,Australia,6,2,7690,15,1,3,blue
2,2,Australia,6,2,7690,15,1,3,blue
